In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from google.colab import drive
import csv
import pandas as pd
import io
import os
import cv2
from pandas.errors import EmptyDataError
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, BatchNormalization, Input 

drive.mount('/drive')

dataset = pd.read_csv('/drive/My Drive/Diabatic_Retinopathy/train.csv',error_bad_lines=False)
image_folder_path = "/drive/My Drive/Diabatic_Retinopathy_Preprocessed_Images"

one_hot_labels = pd.Series(dataset['diagnosis'])
one_hot_labels = pd.get_dummies(one_hot_labels, sparse = True)
one_hot_labels = np.asarray(one_hot_labels)

x = []
y = []

for index, row in dataset.iterrows():
  image_path = os.path.join(image_folder_path, row['id_code']+'.png')
  image = cv2.imread(image_path)
  if image is None:
    print("No Image : ", image_path)
  else: 
    x.append(image)
    y.append(one_hot_labels[index])
    print(index)

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.2, train_size = 0.8, random_state=42)

def build_diabetic_retinopathy_model(kernel_size, pool_size):
  
  diabetic_retinopathy_model = Sequential()

  diabetic_retinopathy_model.add(Conv2D(32, kernel_size, activation='relu', input_shape = (300,300,3)))
  diabetic_retinopathy_model.add(MaxPooling2D((pool_size)))
  diabetic_retinopathy_model.add(Dropout(0.5))

  diabetic_retinopathy_model.add(Conv2D(64, (kernel_size), activation='relu'))
  diabetic_retinopathy_model.add(MaxPooling2D((pool_size)))
  diabetic_retinopathy_model.add(Dropout(0.8))

  diabetic_retinopathy_model.add(Conv2D(128, (kernel_size), activation='relu'))
  diabetic_retinopathy_model.add(MaxPooling2D((pool_size)))
  diabetic_retinopathy_model.add(Dropout(0.8))

  diabetic_retinopathy_model.add(Conv2D(64, (kernel_size), activation='relu'))
  diabetic_retinopathy_model.add(MaxPooling2D((pool_size)))
  diabetic_retinopathy_model.add(Dropout(0.8))

  diabetic_retinopathy_model.add(keras.layers.Flatten())
  diabetic_retinopathy_model.add(keras.layers.Dense(32, activation='relu'))
  diabetic_retinopathy_model.add(keras.layers.Dense(5, activation='softmax'))

  return diabetic_retinopathy_model

kernel_size = 4
pool_size = (2,2)

diabetic_retinopathy_model = build_diabetic_retinopathy_model(kernel_size, pool_size)

diabetic_retinopathy_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

y_train_raw = np.array(y_train)
x_train_raw = np.array(x_train)
x_valid_raw = np.array(x_valid)
y_valid_raw = np.array(y_valid)

history = diabetic_retinopathy_model.fit(x_train_raw, y_train_raw, batch_size = 40, epochs=50, validation_data = (x_valid_raw, y_valid_raw)  )

In [ ]:
# Evaluate Model
score = diabetic_retinopathy_model.evaluate(x_valid_raw, y_valid_raw, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Generating Graphs
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Retrieve a list of list results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

In [ ]:
# Saving the Model
diabetic_retinopathy_model.save('/drive/My Drive/trained_models/approach_1')